<a href="https://colab.research.google.com/github/Aamna-Khan-Git/Models/blob/main/PhonePe_transaction_insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MOUNTING DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# LOADING LIBRARIES

In [ ]:
#!pip install pandas matplotlib seaborn sqlalchemy
import os
import json
import sqlite3
import pandas as pd

import os: This line imports the os module, which provides a way to interact with the operating system, such as accessing files and directories. <br>
import json: This line imports the json module, which is used for working with JSON data (encoding and decoding).<br>
import sqlite3: This line imports the sqlite3 module, which provides an interface for working with SQLite databases.<br>
import pandas as pd: This line imports the pandas library, a powerful tool for data manipulation and analysis, and assigns it the alias pd for convenience.

# DEFINING PATHS

In [ ]:
# Path to your cloned pulse folder in Google Drive
base_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master"

# Ensure the directory exists before connecting to the database
db_path = "/content/drive/MyDrive/Colab Notebooks/labmentix"
os.makedirs(db_path, exist_ok=True)

# SQLite DB file will be saved in Drive
conn = sqlite3.connect(os.path.join(db_path, "phonepe_data.db"))
cursor = conn.cursor()

base_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master": This line sets a variable base_path to the location of your cloned pulse folder in Google Drive. <br>
db_path = "/content/drive/MyDrive/Colab Notebooks/labmentix": This line sets a variable db_path to the directory where you want to store the SQLite database file in Google Drive.<br>
os.makedirs(db_path, exist_ok=True): This line creates the directory specified by db_path if it doesn't already exist. The exist_ok=True argument prevents an error if the directory is already there. <br>
conn = sqlite3.connect(os.path.join(db_path, "phonepe_data.db")): This line establishes a connection to a SQLite database. It uses os.path.join to create the full path to the database file named "phonepe_data.db" within the db_path directory. If the database file doesn't exist, it will be created. The connection object is stored in the conn variable.<br>
cursor = conn.cursor(): This line creates a cursor object from the database connection. A cursor is used to execute SQL commands within the database. The cursor object is stored in the cursor variable.

# CREATING TABLE STRUCTURE

In [ ]:
import sqlite3
import pandas as pd
import json
import os
from pathlib import Path

def create_phonepe_database(db_path="phonepe_data.db"):
    """
    Create PhonePe database with all required tables and populate them with data.

    Args:
        db_path (str): Path to the SQLite database file
    """

    # Connect to the database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    try:
        print("Creating database tables...")

        # Create all tables
        cursor.executescript("""
        -- ========================================
        -- 1. AGGREGATED TABLES
        -- ========================================

        DROP TABLE IF EXISTS aggregated_transaction;
        DROP TABLE IF EXISTS aggregated_user;
        DROP TABLE IF EXISTS aggregated_insurance;

        CREATE TABLE aggregated_transaction (
            state TEXT,
            year INTEGER,
            quarter INTEGER,
            transaction_type TEXT,
            transaction_count INTEGER,
            transaction_amount REAL
        );

        CREATE TABLE aggregated_user (
            state TEXT,
            year INTEGER,
            quarter INTEGER,
            brand TEXT,
            user_count INTEGER,
            percentage REAL
        );

        CREATE TABLE aggregated_insurance (
            state TEXT,
            year INTEGER,
            quarter INTEGER,
            insurance_type TEXT,
            transaction_count INTEGER,
            transaction_amount REAL
        );

        -- ========================================
        -- 2. MAP TABLES
        -- ========================================

        DROP TABLE IF EXISTS map_user;
        DROP TABLE IF EXISTS map_transaction;
        DROP TABLE IF EXISTS map_insurance;

        CREATE TABLE map_user (
            state TEXT,
            district TEXT,
            year INTEGER,
            quarter INTEGER,
            registered_users INTEGER,
            app_opens INTEGER
        );

        CREATE TABLE map_transaction (
            state TEXT,
            district TEXT,
            year INTEGER,
            quarter INTEGER,
            transaction_count INTEGER,
            transaction_amount REAL
        );

        CREATE TABLE map_insurance (
            state TEXT,
            district TEXT,
            year INTEGER,
            quarter INTEGER,
            insurance_type TEXT,
            transaction_count INTEGER,
            transaction_amount REAL
        );

        -- ========================================
        -- 3. TOP TABLES
        -- ========================================

        DROP TABLE IF EXISTS top_user;
        DROP TABLE IF EXISTS top_transaction;
        DROP TABLE IF EXISTS top_insurance;

        CREATE TABLE top_user (
            state TEXT,
            year INTEGER,
            quarter INTEGER,
            pincode TEXT,
            registered_users INTEGER
        );

        CREATE TABLE top_transaction (
            state TEXT,
            year INTEGER,
            quarter INTEGER,
            name TEXT, -- could be district or pincode
            transaction_count INTEGER,
            transaction_amount REAL
        );

        CREATE TABLE top_insurance (
            state TEXT,
            year INTEGER,
            quarter INTEGER,
            insurance_type TEXT,
            transaction_count INTEGER,
            transaction_amount REAL
        );
        """)

        conn.commit()
        print("All tables created successfully.")

        # Load data into tables
        load_aggregated_data(conn)
        load_map_data(conn)
        load_top_data(conn)

        print("Database setup and data loading complete!")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
        conn.rollback()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        conn.close()

import sqlite3, import pandas as pd, import json, import os, from pathlib import Path: These lines import necessary libraries for database operations, data manipulation, JSON handling, file system interactions, and path manipulation.<br>
def create_phonepe_database(db_path="phonepe_data.db"):: This defines the function create_phonepe_database which takes an optional argument db_path for the database file location (defaulting to "phonepe_data.db").<br>
conn = sqlite3.connect(db_path): Establishes a connection to the SQLite database.<br>
cursor = conn.cursor(): Creates a cursor object to execute SQL commands.
try...except...finally: This block handles potential errors during database operations.<br>
cursor.executescript(...): This executes a multi-line SQL script.<br>
DROP TABLE IF EXISTS ...: These lines drop the tables if they already exist, ensuring a clean creation each time the script is run.<br>
CREATE TABLE ...: These lines create the various tables (aggregated_transaction, aggregated_user, aggregated_insurance, map_user, map_transaction, map_insurance, top_user, top_transaction, top_insurance) with their respective columns and data types.<br>
conn.commit(): Saves the changes made by the CREATE TABLE statements to the database.<br>
load_aggregated_data(conn), load_map_data(conn), load_top_data(conn): These lines call other functions (presumably defined elsewhere in the notebook) to load data into the newly created tables.<br>
conn.rollback(): If an error occurs within the try block, this rolls back any changes made since the last commit, preventing inconsistent data.<br>
conn.close(): Closes the database connection in the finally block, ensuring it's always closed even if errors occur.<br>


# LOADING DATA FROM DRIVE TO THE AGGREGATED TABLES

In [ ]:
def load_aggregated_data(conn):
    """Load data into aggregated tables"""
    print("\nLoading AGGREGATED data...")

    # AGGREGATED TRANSACTION DATA
    print("  Loading aggregated_transaction...")
    aggregated_transaction_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master/data/aggregated/transaction"

    # Example code to load JSON files from the aggregated transaction directory
    for state_folder in os.listdir(aggregated_transaction_path):
        state_path = os.path.join(aggregated_transaction_path, state_folder)
        if os.path.isdir(state_path):
            for year_folder in os.listdir(state_path):
                year_path = os.path.join(state_path, year_folder)
                if os.path.isdir(year_path):
                    for quarter_file in os.listdir(year_path):
                        if quarter_file.endswith('.json'):
                            file_path = os.path.join(year_path, quarter_file)
                            with open(file_path, 'r') as f:
                                data = json.load(f)
                                # Process and insert data into aggregated_transaction table
                                process_aggregated_transaction_data(conn, data, state_folder, year_folder, quarter_file)

    # AGGREGATED USER DATA
    print("  Loading aggregated_user...")
    aggregated_user_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master/data/aggregated/user"

    # Example code to load user data

    for state_folder in os.listdir(aggregated_user_path):
        state_path = os.path.join(aggregated_user_path, state_folder)
        if os.path.isdir(state_path):
            for year_folder in os.listdir(state_path):
                year_path = os.path.join(state_path, year_folder)
                if os.path.isdir(year_path):
                    for quarter_file in os.listdir(year_path):
                        if quarter_file.endswith('.json'):
                            file_path = os.path.join(year_path, quarter_file)
                            with open(file_path, 'r') as f:
                                data = json.load(f)
                                # Process and insert data into aggregated_user table
                                process_aggregated_user_data(conn, data, state_folder, year_folder, quarter_file)


    # AGGREGATED INSURANCE DATA
    print("  Loading aggregated_insurance...")
    aggregated_insurance_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master/data/aggregated/insurance"

    # Example code to load insurance data

    for state_folder in os.listdir(aggregated_insurance_path):
        state_path = os.path.join(aggregated_insurance_path, state_folder)
        if os.path.isdir(state_path):
            for year_folder in os.listdir(state_path):
                year_path = os.path.join(state_path, year_folder)
                if os.path.isdir(year_path):
                    for quarter_file in os.listdir(year_path):
                        if quarter_file.endswith('.json'):
                            file_path = os.path.join(year_path, quarter_file)
                            with open(file_path, 'r') as f:
                                data = json.load(f)
                                # Process and insert data into aggregated_insurance table
                                process_aggregated_insurance_data(conn, data, state_folder, year_folder, quarter_file)


    print("Aggregated data loading complete.")

def load_aggregated_data(conn):: This defines the function load_aggregated_data which takes the database connection object (conn) as an argument.<br>
print("\nLoading AGGREGATED data..."): Prints a message indicating the start of aggregated data loading. <br>
AGGREGATED TRANSACTION DATA: This section handles loading data into the aggregated_transaction table.<br>
aggregated_transaction_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master/data/aggregated/transaction": Sets the path to the aggregated transaction data in your Google Drive.<br>
for state_folder in os.listdir(aggregated_transaction_path):, for year_folder in os.listdir(state_path):, for quarter_file in os.listdir(year_path):: These nested loops iterate through the state, year, and quarter folders/files within the specified path.<br>
if quarter_file.endswith('.json'):: Checks if the file is a JSON file.
file_path = os.path.join(year_path, quarter_file): Constructs the full path to the JSON file.<br>
with open(file_path, 'r') as f:: Opens the JSON file for reading.
data = json.load(f): Loads the JSON data from the file into the data variable.<br>
process_aggregated_transaction_data(conn, data, state_folder, year_folder, quarter_file): This line calls another function (presumably defined elsewhere) to process the loaded JSON data and insert it into the aggregated_transaction table. The state, year, and quarter information are also passed to this function.<br>
AGGREGATED USER DATA and AGGREGATED INSURANCE DATA: These sections follow a similar structure to the aggregated transaction data section, loading data into the aggregated_user and aggregated_insurance tables respectively, using the process_aggregated_user_data and process_aggregated_insurance_data functions.<br>
print("Aggregated data loading complete."): Prints a message indicating the completion of aggregated data loading.

# LOADING DATA FROM DRIVE TO MAP TABLES

In [ ]:
def load_map_data(conn):
    """Load data into map tables"""
    print("\nLoading MAP data...")

    # MAP USER DATA
    print("  Loading map_user...")
    map_user_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master/data/map/user"

    # Example code to load map user data

    for state_folder in os.listdir(map_user_path):
        state_path = os.path.join(map_user_path, state_folder)
        if os.path.isdir(state_path):
            for year_folder in os.listdir(state_path):
                year_path = os.path.join(state_path, year_folder)
                if os.path.isdir(year_path):
                    for quarter_file in os.listdir(year_path):
                        if quarter_file.endswith('.json'):
                            file_path = os.path.join(year_path, quarter_file)
                            with open(file_path, 'r') as f:
                                data = json.load(f)
                                # Process and insert data into map_user table
                                process_map_user_data(conn, data, state_folder, year_folder, quarter_file)


    # MAP TRANSACTION DATA
    print("  Loading map_transaction...")
    map_transaction_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master/data/map/transaction"

    # Example code to load map transaction data

    for state_folder in os.listdir(map_transaction_path):
        state_path = os.path.join(map_transaction_path, state_folder)
        if os.path.isdir(state_path):
            for year_folder in os.listdir(state_path):
                year_path = os.path.join(state_path, year_folder)
                if os.path.isdir(year_path):
                    for quarter_file in os.listdir(year_path):
                        if quarter_file.endswith('.json'):
                            file_path = os.path.join(year_path, quarter_file)
                            with open(file_path, 'r') as f:
                                data = json.load(f)
                                # Process and insert data into map_transaction table
                                process_map_transaction_data(conn, data, state_folder, year_folder, quarter_file)


    # MAP INSURANCE DATA
    print("  Loading map_insurance...")
    map_insurance_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master/data/map/insurance"

    # Example code to load map insurance data

    for state_folder in os.listdir(map_insurance_path):
        state_path = os.path.join(map_insurance_path, state_folder)
        if os.path.isdir(state_path):
            for year_folder in os.listdir(state_path):
                year_path = os.path.join(state_path, year_folder)
                if os.path.isdir(year_path):
                    for quarter_file in os.listdir(year_path):
                        if quarter_file.endswith('.json'):
                            file_path = os.path.join(year_path, quarter_file)
                            with open(file_path, 'r') as f:
                                data = json.load(f)
                                # Process and insert data into map_insurance table
                                process_map_insurance_data(conn, data, state_folder, year_folder, quarter_file)


    print("Map data loading complete.")


def load_map_data(conn):: This defines the function load_map_data which takes the database connection object (conn) as an argument.<br>
print("\n Loading MAP data..."): Prints a message indicating the start of map data loading.<br>
MAP USER DATA: This section handles loading data into the map_user table.
map_user_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master/data/map/user": Sets the path to the map user data in your Google Drive.<br>
The nested loops and file handling (os.listdir, os.path.join, endswith('.json'), open, json.load) work similarly to the load_aggregated_data function, iterating through state, year, and quarter folders/files to find JSON files.<br>
process_map_user_data(conn, data, state_folder, year_folder, quarter_file): This line calls a function to process the loaded JSON data and insert it into the map_user table, passing the connection, data, state, year, and quarter information.<br>
MAP TRANSACTION DATA and MAP INSURANCE DATA: These sections follow the same pattern as the map user data section, loading data into the map_transaction and map_insurance tables respectively, using the process_map_transaction_data and process_map_insurance_data functions.<br>
print(" Map data loading complete."): Prints a message indicating the completion of map data loading.

# LOADING DATA FROM DRIVE TO TOP TABLES

In [ ]:
def load_top_data(conn):
    """Load data into top tables"""
    print("\n Loading TOP data...")

    # TOP USER DATA
    print("  Loading top_user...")
    top_user_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master/data/top/user"

    # Example code to load top user data

    for state_folder in os.listdir(top_user_path):
        state_path = os.path.join(top_user_path, state_folder)
        if os.path.isdir(state_path):
            for year_folder in os.listdir(state_path):
                year_path = os.path.join(state_path, year_folder)
                if os.path.isdir(year_path):
                    for quarter_file in os.listdir(year_path):
                        if quarter_file.endswith('.json'):
                            file_path = os.path.join(year_path, quarter_file)
                            with open(file_path, 'r') as f:
                                data = json.load(f)
                                # Process and insert data into top_user table
                                process_top_user_data(conn, data, state_folder, year_folder, quarter_file)


    # TOP TRANSACTION DATA
    print("  Loading top_transaction...")
    top_transaction_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master/data/top/transaction"

    # Example code to load top transaction data

    for state_folder in os.listdir(top_transaction_path):
        state_path = os.path.join(top_transaction_path, state_folder)
        if os.path.isdir(state_path):
            for year_folder in os.listdir(state_path):
                year_path = os.path.join(state_path, year_folder)
                if os.path.isdir(year_path):
                    for quarter_file in os.listdir(year_path):
                        if quarter_file.endswith('.json'):
                            file_path = os.path.join(year_path, quarter_file)
                            with open(file_path, 'r') as f:
                                data = json.load(f)
                                # Process and insert data into top_transaction table
                                process_top_transaction_data(conn, data, state_folder, year_folder, quarter_file)


    # TOP INSURANCE DATA
    print("  Loading top_insurance...")
    top_insurance_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master/data/top/insurance"

    # Example code to load top insurance data

    for state_folder in os.listdir(top_insurance_path):
        state_path = os.path.join(top_insurance_path, state_folder)
        if os.path.isdir(state_path):
            for year_folder in os.listdir(state_path):
                year_path = os.path.join(state_path, year_folder)
                if os.path.isdir(year_path):
                    for quarter_file in os.listdir(year_path):
                        if quarter_file.endswith('.json'):
                            file_path = os.path.join(year_path, quarter_file)
                            with open(file_path, 'r') as f:
                                data = json.load(f)
                                # Process and insert data into top_insurance table
                                process_top_insurance_data(conn, data, state_folder, year_folder, quarter_file)


    print("Top data loading complete.")

def load_top_data(conn):: This defines the function load_top_data which takes the database connection object (conn) as an argument.<br>
print("\n Loading TOP data..."): Prints a message indicating the start of top data loading.<br>
TOP USER DATA: This section handles loading data into the top_user table.
top_user_path = "/content/drive/MyDrive/Colab Notebooks/labmentix/pulse-master/data/top/user": Sets the path to the top user data in your Google Drive.<br>
The nested loops and file handling (os.listdir, os.path.join, endswith('.json'), open, json.load) work similarly to the previous loading functions, iterating through state, year, and quarter folders/files to find JSON files.<br>
process_top_user_data(conn, data, state_folder, year_folder, quarter_file): This line calls a function to process the loaded JSON data and insert it into the top_user table, passing the connection, data, state, year, and quarter information.<br>
TOP TRANSACTION DATA and TOP INSURANCE DATA: These sections follow the same pattern as the top user data section, loading data into the top_transaction and top_insurance tables respectively, using the process_top_transaction_data and process_top_insurance_data functions.<br>
print("Top data loading complete."): Prints a message indicating the completion of top data loading.<br>
(process_aggregated_transaction_data, process_aggregated_user_data, etc.) that are called within these loading functions to actually parse the JSON data and insert it into the database tables. Once those are defined, you can call the create_phonepe_database() function to create the database and load all the data.

# DISPLAYING TOP 10 STATES

In [ ]:
import pandas as pd

# SQL query to get top 10 states by total transaction amount
query = """
SELECT state,
       SUM(transaction_amount) AS total_amount,
       SUM(transaction_count) AS total_transactions
FROM aggregated_transaction
GROUP BY state
ORDER BY total_amount DESC
LIMIT 10;
"""

# Run the query
df_top_states = pd.read_sql(query, conn)
df_top_states


DatabaseError: Execution failed on sql '
SELECT state, 
       SUM(transaction_amount) AS total_amount, 
       SUM(transaction_count) AS total_transactions
FROM aggregated_transaction
GROUP BY state
ORDER BY total_amount DESC
LIMIT 10;
': no such table: aggregated_transaction